In [ ]:
import torch
import torchmetrics.classification
from torch.utils.data import DataLoader
from torchmetrics import ConfusionMatrix

from dataset import get_dataset
from resnet_module import ResNetModule
import pytorch_lightning as pl

import matplotlib.pyplot as plt
import seaborn as sns

from statistics import variance

In [ ]:
# Evaluate overall accuracy

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

model = ResNetModule.load_from_checkpoint('resnet18-1.cpkt', map_location=device)

trainer = pl.Trainer(
    accelerator="auto",
    devices=1,
    logger=False,
)

data_set_with_metadata = get_dataset("test", include_metadata=True)
data_loader = DataLoader(data_set_with_metadata, batch_size=1, shuffle=False, drop_last=False, num_workers=1)

all_labels = []
for batch in data_loader:
    imgs, labels = batch
    all_labels.append(labels)

print(all_labels)
predictions = trainer.predict(model, data_loader)
predictions = torch.cat(predictions)
print(predictions)

class_labels = []
for label in all_labels:
    class_labels.append(label[0])
class_labels = torch.cat(class_labels)
metric = torchmetrics.classification.MulticlassAccuracy(num_classes=4, average='weighted')
overall_accuracy = metric(predictions, class_labels)
print(f"overall weighted accuracy: {overall_accuracy}")

In [ ]:
# Plot confusion matrix

confm = ConfusionMatrix(task="multiclass", num_classes=4)
result = confm(predictions, class_labels)

labels = ["MEL", "NV", "BCC", "BKL"]
sns.heatmap(result, annot=True, cmap="Blues", xticklabels=labels, yticklabels=labels)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.savefig("conf_ma.png")

In [ ]:
# Evaluate bias based on sex

male_predictions = []
male_labels = []
female_predictions = []
female_labels = []

for i in range(len(predictions)):
    if all_labels[i][2][0] == 'male':
        male_predictions.append(torch.unsqueeze(predictions[i], dim=0))
        male_labels.append(all_labels[i][0])
    elif all_labels[i][2][0] == 'female':
        female_predictions.append(torch.unsqueeze(predictions[i], dim=0))
        female_labels.append(all_labels[i][0])
    else:
        print(f"unknown: {all_labels[i][2]}")

print(male_labels)
print(male_predictions)

metric = torchmetrics.classification.MulticlassAccuracy(num_classes=4, average='weighted')

male_accuracy = metric(torch.cat(male_predictions), torch.cat(male_labels))
female_accuracy = metric(torch.cat(female_predictions), torch.cat(female_labels))

print(f"male_acc: {male_accuracy}")
print(f"female_acc: {female_accuracy}")
print(f"bias: {variance([male_accuracy.item(), female_accuracy.item()])}")

In [ ]:
# Evaluate bias based on age groups

age_groups = ["upto30", "35to55", "60up", "unknown"]
age_based_predictions = {"upto30": [], "35to55": [], "60up": [], "unknown": []}
age_labels = {"upto30": [], "35to55": [], "60up": [], "unknown": []}

for i in range(len(predictions)):
    age = all_labels[i][1][0].item()
    if age <= 0.0:
        age_group = "unknown"
    if age <= 30.0:
        age_group = "upto30"
    elif age_group <= 55.0:
        age_group = "35to55"
    else:
        age_group = "60up"
    
    age_based_predictions[age_group].append(torch.unsqueeze(predictions[i], dim=0))
    age_labels[age_group].append(all_labels[i][0])

accuracies = {}
acc_list = []
for age_group in age_groups:
    print(f"{age_group} has {len(age_based_predictions[age_group])} samples")
    if age_group == "unknown":
        continue
    if len(age_based_predictions[age_group]) > 0:
        accuracies[age_group] = metric(torch.cat(age_based_predictions[age_group]), torch.cat(age_labels[age_group])).item()
        acc_list.append(accuracies[age_group])
    else:
        print("WARNING: No samples for this age group")

print(f"accuracies: {accuracies}")
print(f"bias: {variance(acc_list)}")


## Resnet without new sample probs

upto30 has 153 samples
35to55 has 606 samples
60up has 631 samples
-10.0 has 0 samples
{'upto30': 0.24836601316928864, '35to55': 0.6716171503067017, '60up': 0.48335975408554077}
overall_accuracy: 0.539568305015564
bias: 0.0449674050983165

## Resnet with new sample probs

upto30 has 153 samples
35to55 has 606 samples
60up has 631 samples
-10.0 has 0 samples
{'upto30': 0.32679739594459534, '35to55': 0.6881188154220581, '60up': 0.4358161687850952}
overall_accuracy: 0.533812940120697
bias: 0.034349147750812214



## Resnet without new sample probs

upto30 has 153 samples
35to55 has 606 samples
60up has 631 samples
-10.0 has 0 samples
{'upto30': 0.24836601316928864, '35to55': 0.6716171503067017, '60up': 0.48335975408554077}
overall_accuracy: 0.539568305015564
bias: 0.0449674050983165

## Resnet with new sample probs

upto30 has 153 samples
35to55 has 606 samples
60up has 631 samples
-10.0 has 0 samples
{'upto30': 0.32679739594459534, '35to55': 0.6881188154220581, '60up': 0.4358161687850952}
overall_accuracy: 0.533812940120697
bias: 0.034349147750812214
